We load the packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.tri as mtri

Then, one defines the domain here. I think the easiest way to do this is to hardcode the domain once, then refine it.

1. square
2. triangle
3. L-shaped

I used a domain of size 1 in x, that I can easily scale be dividing by the number of elements in x (L) of size h such that L\*h=1 always. However there are (L+1)^2 nodes here for a square. There are therefore L\*H elements in this construction.

Then, the array coordinates is of shape ((L+1)\*(H+1), 2), where each row is (x-position, y-position). I try to go anticlockwise for everynodes.

In [ ]:
#number of elements in x and y
L=2
H=2
coordinates1=np.zeros(((L+1)*(H+1),2))
i=0
while True:
    coordinates1[i,0]=(i%(L+1))/L
    coordinates1[i,1]=i//(L+1)/L
    i+=1
    if i==(L+1)*(H+1):
        break

Now, it is time for the elements. So every element is defined counterclockwise. The array elements3 is of shape (2\*L\*H,3), where each row is like this (node1, node2, node3).

In [ ]:
"""
elements3=np.zeros((2*L*H,4))
i=0
while True:
    elements3[i,0]=i
    if (i-(i//(L)))%(L)==0:
        elements3[i,1]=i-(i//(L+1))
    if (i-(i//(L)))%(L)!=0:
        elements3[i,1]=0
    elements3[i,2]=(i+1)%2+i%2*(L+1)
    elements3[i,3]=1
    i+=1
    if i==L*H*2:
        break
"""
elements3=np.array([[0,1,3],[1,4,3],[1,2,4],[2,5,4],[3,4,6],[4,7,6],[4,5,7],[5,8,7]])

Suppose now that the given square is such that on the sides it is the Neumann BC and on the top and bottom the Dirichlet BC. Both are size (2\*H,2) and (2\*L,2) respectively.

In [ ]:
"these are triangle 0, 3, 4, 7 resp." 
neumannBC=np.array([[0,3],[2,5],[3,6],[5,8]])
"these are triangle 0, 2, 5, 7 resp."
dirichletBC=np.array([[0,1],[1,2],[6,7],[7,8]])

The next function is there in order to compute all the different part of the stiffness matrix (it is the M matrix in the manual).

We also define the force function f. This is whatever we want it to be.

And also the stress function g. This is for Neumann BC.

And also the Dirichlet BC.

And also the show function.

In [ ]:
def stima3(vertices):
    "function that takes in argument the vertices of a triangle and gives the corresponding stiffness\
    matrix part"
    d=np.size(vertices,1)
    G=np.linalg.lstsq(np.concatenate((np.ones((1,d+1)),vertices.T),axis=0),\
                      np.concatenate((np.zeros((1,d)),np.eye(d))))
    G=np.around(G[0])
    M=np.linalg.det(np.concatenate((np.ones((1,d+1)),vertices.T),axis=0))\
    *(G@G.T)*np.prod(np.arange(1,d))
    return M

def f(x):
    "Insert here the proper function that one wants"
    return np.ones((np.size(x,axis=0)+1,1))

def g(x):
    "Insert here the proper function that one wants"
    return np.zeros((np.size(x,axis=0),1))

def u_d(x):
    "Insert here the proper function that one wants"
    return np.zeros((np.size(x,axis=0),1))

def show(elements3,coordinates1,u):
    fig=plt.figure()
    ax=fig.gca(projection='3d')
    ax.plot_trisurf(coordinates1[:,0],coordinates1[:,1],u.T)
    return plt.show()
    

Now we define the free nodes which will reprensent the nodes not being forced to be something by Dirichlet. Also we define the stiffness matrix A and the output vector b.

Then we can assemble our stiffness mastrix by a simple for loop.

In [ ]:
FreeNodes=np.setdiff1d(np.arange(0,np.size(coordinates1,0)),np.unique(dirichletBC,axis=0))
A=np.zeros((np.size(coordinates1,0),np.size(coordinates1,0)))
b=np.zeros((np.size(coordinates1,0),1))
u=np.zeros((np.size(coordinates1,0),1))
for j in np.arange(0,np.size(elements3,0)):
    #assembly of stiffness matrix
    A[np.ix_(elements3[j],elements3[j])]+=stima3(coordinates1[elements3[j]])
    #volume force
    b[np.ix_(elements3[j])]+=np.linalg.det(np.concatenate((np.ones((1,3)),coordinates1[elements3[j]].T),axis=0))\
    *f(np.sum(coordinates1[elements3[j]],axis=0)/3)/6

Then we have to deal with the boundary conditions. First Neumann, than Dirichlet:

In [ ]:
#Neumann
for j in np.arange(0,np.size(neumannBC,0)):
    b[np.ix_(neumannBC[j])]+=np.sqrt((coordinates1[neumannBC[j,0]]-coordinates1[neumannBC[j,1]])@\
                                    (coordinates1[neumannBC[j,0]]-coordinates1[neumannBC[j,1]]))\
    *g(np.sum(coordinates1[neumannBC[j]],axis=0)/2)/2

#Dirichlet
'The error here is somehow in the evalution of u'
#u[(np.unique(dirichletBC,axis=0))]=u_d(coordinates1[np.unique(dirichletBC,axis=0)])
b-=A@u

Then, all is left is to compute the solution:

In [ ]:
u[FreeNodes]=np.linalg.lstsq(A[np.ix_(FreeNodes,FreeNodes)],b[FreeNodes])[0]
#show(elements3,coordinates1,u)

For some reason, my 'show' function doesn't work... Also, I could not implement Dirichlet BC, altought I was pretty close to do it. This program should handle properly any domain. Here is it is a simple square  which is subdivised into 4 sqaures that are then split into 2 triangles each. But any triangulation, of any wierd shape domain should work, as long as the format of the coordinates and elements3 works. This code correctly computes the solution for the Laplace problem.

As for the bissection implementation, I honestly don't know how to do this. I looked everywhere, read everybit I could find, and I just was not able to do it. I would like to see it however. And from that, I could not do the numerical experiments on finer mesh.